CS524: Introduction to Optimization Lecture 4
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 9, 2023
--------------

## Recall: TopBrass Algebraic Model

$$\max z$$
subject to
\begin{eqnarray*}
12 x_1 + 9 x_2 & = & z \qquad \mbox{(Define Profit)}\\
x_1 & \leq & 1000 \qquad \mbox{(Football inventory)}\\
x_2 & \leq & 1500 \qquad \mbox{(Soccer ball inventory)}\\
x_1 + x_2 & \leq & 1750 \qquad \mbox{(Plaque inventory)}\\
4 x_1 + 2 x_2 & \leq & 4800 \qquad \mbox{(Wood inventory)}\\
x_1, x_2 & \geq & 0 \qquad \mbox{(Antimatter inventory)}
\end{eqnarray*}

--------------

## BigTopBrass

### We're Going Big Time!
- A General Goal: We'd like to think of an abstract model
----------------------------------

- What if instead of 2 products, we had 20,000.
- What if instead of 3 resources required to make the products, we had 3000?

----------

|Symbol| Description|
|---|---|
| $I$ | Set of items (types of trophies) to sell |
| $R$ | Set of resources requires to make items |
| $c_i$ | Per-unit profit of producing $i \in  I$ |
| $u_i$ | Maximum number of $i \in I$ to produce |
| $b_r$ | Available Amt. of resource $r \in R$ |
| $a_{ri}$ | Amount of resource $r \in R$ required to produce one $i \in I$ |

----------

## BigTopBrass

$$ \max \sum_{i \in I}{c_ix_i}$$

s.t.

$$\sum_{i \in I}{a_{ri}  x_i} \leq b_r,  \forall_r \in R$$
$$ x_i \leq u_i,  \forall_i \in I$$
$$ x_i \geq 0,  \forall_i \in I$$

--- 

Matrix Form: 
$$ \max_{0 \leq x \leq u} {\{ c^Tx | Ax \leq b \}} $$

---

### Next steps
- Learn how to put this 'symbolic' model into GAMS - using sets!
- Separate data, model and post processing
- Use an exchange container to move data to/from GAMS

In order to use GAMS, we have to load GAMS extension. Running the next script will load it and set up exchange container.

In [1]:
%load_ext gams.magic
m = gams.exchange_container

We show how to do pre/post processing in python, with only the model solved in GAMS.

### GAMS tip

Right now, we will declare a set and a new decision variable on that set. You can do it by running the following code.


In [2]:
%%gams
set I /football,soccer/;
set R /plaques,wood/;

table a(R,I)  "Per-Unit resource requirements"
         football   soccer
plaques     1         1
wood        4         2  ;

parameters
    c(I) / "football" 12, "soccer" 9 /
    u(I) / "football" 1000 , "soccer"  1500 /
    b(R) / "plaques"  1750,  "wood"  4800 /;

* VARIABLE AND EQUATION DECLARATIONS
free variable profit "total profit";

positive variables
x(I)     "number trophies" ;

c:\Users\Rocke\anaconda3\envs\gams\Lib\site-packages\gams\transfer\syms\_mixins\spve.py:1006: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(self.records[i]):
c:\Users\Rocke\anaconda3\envs\gams\Lib\site-packages\gams\transfer\syms\_mixins\spve.py:1006: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(self.records[i]):
c:\Users\Rocke\anaconda3\envs\gams\Lib\site-packages\gams\transfer\syms\_mixins\spve.py:1006: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(self.records[i]):
c:\Users\Rocke\anaconda3\envs\gams\Lib\site-packages\gams\transfer\syms\_mixins\spve.py:1006: FutureWarning: is_categorical_dtype is deprecated and will be r

Alternatively, you can use the following to generate the same data

In [3]:
# DATA
import numpy as np

i = m.addSet('I',records=['football','soccer'])
r = m.addSet('R',records=['plaques','wood'])
c = m.addParameter('c',[i],records=np.array([12., 9.]))
b = m.addParameter('b',[r],records=np.array([1750., 4800.]))
a = m.addParameter('a',[r,i],records=np.array([[1., 1.], [4., 2.]]),description="Per-Unit resource requirements")
                    
profit = m.addVariable('profit','free',description="total profit")
x = m.addVariable('x','positive',[i],description="number trophies",records={"upper": np.array([1000., 1500.])})
#m.addEquation('profit_eq','e',description="profit definition")
#m.addEquation('resource_con','l',[r],description="resource limit");
#m.write('tbdata.gdx')

### MODEL: topbrass4.gms
- Now note that we can use $\sum_{i\in I}$ notation in GAMS
- Following code must be in GAMS
- Suggest debugging this in GAMS Studio and then copy/pasting into Jupyter cell

In [4]:
%%gams
equation 
    profit_eq "profit definition",
    resource_con(R) "resource limit";

* EQUATION (MODEL) DEFINITION
profit_eq..
  profit =E= sum(I,c(I)*x(I));

resource_con(R)..
  sum(I, a(R,I)*x(I)) =L= b(R);

model btb /all/;

solve btb using lp maximizing profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),17700.0,3,3,LP,CPLEX,0.021


While declaring variables, if you redeclare the variable with another domain(set or sets), it will throw an exception. 

# POST-PROCESSING: 
Let's pull the solution and see x

In [5]:
# x = m.data['x']
display(x.records)

,I,level,marginal,lower,upper,scale
0,football,650.0,0.0,0.0,1000.0,1.0
1,soccer,1100.0,0.0,0.0,1500.0,1.0


- You may have noticed that we have a new column, representing the set X is declared on, added to table.

### Display Statements and Postprocessing
- After the solve statement, one may not want to display the final values of the variables but rather some “processed” form.
- Suppose we wanted to know the final percentage of footballs and soccer balls in the final total.

In [6]:
%%gams
alias (I,J);
parameter pct(I);
pct(I) = 100*x.l(I) / sum(J,x.l(J));
display pct;

- Note the use of alias (I,J);—I is already “under control”, so it cannot be used again
- Another alternative for postprocessing is Python.

In [7]:
gams.gams_lst('-e')
display(m.data['pct'].records.set_index('I'))

# convert x into a dense numpy.array format
xL = m.data['x'].toDense()
display(xL/np.sum(xL))
# display(m.data['pct'].toDict() ) # or toList() or toDense()

E x e c u t i o n


----     66 PARAMETER pct  

football 37.143,    soccer   62.857





,value
I,
football,37.142857
soccer,62.857143


array([0.37142857, 0.62857143])

### GAMS Functions

- GAMS has functions. See Section 6.3.3.
    - `uniform(x,y)`: Returns uniform number between x and y     
    - `round(x)`: Round x to the nearest integer
    - ...

Try it in gams (bigtopbrass.gms) or in python as follow:

In [8]:
rng = np.random.default_rng(seed=123)

i = m.addSet('I',records=['t'+str(v+1) for v in range(0,1000)])
r = m.addSet('R',records=['r'+str(v+1) for v in range(0,500)])
c = m.addParameter('c',[i],records=rng.integers(2,20,1000,endpoint=True),description="Profit margin")
b = m.addParameter('b',[r],records=rng.integers(5000,25000,500,endpoint=True),description="Amt. available")
a = m.addParameter('a',[r,i],records=rng.integers(1,5,(500, 1000),endpoint=True),description="Per-Unit resource requirements")
                    
profit = m.addVariable('profit','free',description="total profit")
x = m.addVariable('x','positive',[i],description="number trophies",records={"upper": rng.integers(500,2500,1000,endpoint=True)})

In [9]:
gams.gams('solve btb using lp maximizing profit')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),39958.3778,501,1001,LP,CPLEX,0.135


### Handy Debugging Tips
- For help debugging the model, don’t set <br> `option limrow = 0, limcol = 0;` <br> GAMS will list the equations for you, up to the value of limrow and limcol. (Default 3) 
- Equations are rearranged algebraically to put all constant terms on one side of the equality or inequality
- Current “.l” values are substituted into the “variable” side and the value of this side is printed along with feasibility/infeasibility.
- GAMS usually doesn’t list zeros - replaces them with “.” or ignores them completely

In [10]:
# %gams_cleanup --closedown
gams.gams_cleanup('--closedown')